# 02. 텍스트 데이터 다루기

## 2.2 텍스트 토큰화하기

In [20]:
! uv pip install tiktoken

Using Python 3.12.11 environment at: /Users/hhkim/Study/LLMs-from-scratch-hands-on/.venv
Audited 1 package in 3ms


In [21]:
import urllib.request
url = ("https://raw.githubusercontent.com/rickiepark/"
	"llm-from-scratch/main/ch02/01_main-chapter-code/"
	"the-verdict.txt")
file_path = "the-verdict.txt"
urllib.request.urlretrieve(url, file_path)

('the-verdict.txt', <http.client.HTTPMessage at 0x11f1ea570>)

In [22]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
	raw_text = f.read()
print("총 문자 개수:", len(raw_text))
print(raw_text[:99])

총 문자 개수: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [23]:
import re  # 파이썬 정규표현식 라이브러리
text = "Hello, world. This, is a test."
result = re.split(r'(\s)', text )  # 공백을 기준으로 텍스트 분할
print(result)

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


In [24]:
result = re.split(r'([,.]|\s)', text )  # , . 또는 공백을 기준으로 텍스트 분할
print(result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


In [25]:
result = [item for item in result if item.strip()]  # 공백 삭제
print(result)

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']


In [26]:
# 다양한 특수 문자 토큰화 테스트
text = "Hello, world. Is this-- a test?"
result = re.split(r'([,.:;?_!"()\']|--|\s)', text)  # text 토큰화
result = [item.strip() for item in result if item.strip()]  # 공백 삭제
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


In [27]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)  # the-verdict 소설 토큰화
preprocessed = [item.strip() for item in preprocessed if item.strip()]  # 공백 삭제
print(len(preprocessed))  # 공백을 제외한 텍스트에 있는 토큰의 개수

4690


In [28]:
print(preprocessed[:30])  # 처음 30개 토큰 출력

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


## 2.3 토큰을 토큰 ID로 변환하기

In [29]:
# 어휘 사전 구축
all_words = sorted(set(preprocessed))  # set으로 중복 제거 후 알파벳 순 정렬
vocab_size = len(all_words)  # 어휘 사전의 개수
print(vocab_size)

1130


In [30]:
vocab = {token:integer for integer, token in enumerate(all_words)}  # 어휘 사전에서 token을 int(ID)와 짝지어 딕셔너리 저장
for i, item in enumerate(vocab.items()):  # 딕셔너리에서 순서대로 51개 출력
	print(item)
	if i >= 50:
		break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


In [31]:
# 어휘 사전을 새로운 텍스트에 적용하여 토큰 ID로 변환
class SimpleTokenizerV1:
	def __init__(self, vocab):
		self.str_to_int = vocab  # encode 메서드와 decode 메서드에서 참조할 수 있도록 어휘사전을 클래스의 속성으로 저장
		self.int_to_str = {i:s for s,i in vocab.items()}  # 토큰 ID를 원본 텍스트 토큰으로 매핑하는 역어휘사전 생성

	# 입력 텍스트를 처리하여 토큰 ID로 변경
	def encode(self, text):
		preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)  # 입력받은 text를 토큰화
		preprocessed = [  # 공백 삭제
			item.strip() for item in preprocessed if item.strip()
		]
		ids = [self.str_to_int[s] for s in preprocessed]  # 토큰화 된 텍스트에서 한 단어씩 어휘사전에서 검색후 id 반환
		return ids
	
	# 토큰 ID를 텍스트로 되돌리기
	def decode(self, ids):
		text = " ".join([self.int_to_str[i] for i in ids])  # text에 ids에서 정수를 하나씩 역어휘사전에서 검색 후 str로 변환해서 추가함(공백으로 구분)
		# return text
		# 지정된 구두점 문자 앞의 공백 삭제
		text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)  # \s과 캡처그룹()이 함께 있으면 캡처그룹만 남겨서 공백을 삭제하는 정규식
		return text

In [32]:
tokenizer = SimpleTokenizerV1(vocab)  # 토크나이저 객체 생성
text = """"It's the last he painted, you know," Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)  # 토큰화
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [33]:
print(tokenizer.decode(ids))  # 토큰 ID를 다시 텍스트로 변경

" It' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.


In [34]:
text = "Hello, do you like tea?"  # 어휘사전에 없는 단어(Hello) 검색
print(tokenizer.encode(text))

KeyError: 'Hello'

## 2.4 특수 문맥 토큰 추가하기

In [35]:
all_tokens = sorted(list(set(preprocessed)))  # 토큰화한 verdict 소설을 set으로 중복 제거 후 list로 만들어 정렬
all_tokens.extend(["<|endoftext|>", "<|unk|>"])  # 토큰 추가
vocab = {token:integer for integer,token in enumerate(all_tokens)}  # 어휘사전 생성

print(len(vocab.items()))  # 토큰 2개 추가된 1132개

1132


In [36]:
for i,item in enumerate(list(vocab.items())[-5:]):  # 어휘사전에서 마지막 5개 출력
	print(item)  # endoftext, unk 토큰 확인

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


In [37]:
# 모르는 단어를 처리하는 토크나이저
class SimpleTokenizerV2:
	def __init__(self, vocab):
		self.str_to_int = vocab  # 어휘사전
		self.int_to_str = { i:s for s,i in vocab.items() }  # 역어휘사전

	def encode(self, text):
		preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)  # text 토큰화
		preprocessed = [ item.strip() for item in preprocessed if item.strip() ]  # 공백 제거
		preprocessed = [ item if item in self.str_to_int else "<|unk|>" for item in preprocessed ]  # 모르는 단어를 unk 토큰으로 변환

		ids = [ self.str_to_int[s] for s in preprocessed ]  # 단어 토큰을 ID로 변환
		return ids

	def decode(self, ids):
		text = " ".join([self.int_to_str[i] for i in ids])  # ID를 토큰으로 변환 (공백으로 구분)
		text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)  # 공백+구두점으로 되어있으면 공백 삭제
		return text

In [38]:
text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."
text = " <|endoftext|> ".join((text1, text2))  # endoftext 토큰으로 구분해서 text 합치기
print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [39]:
tokenizer = SimpleTokenizerV2(vocab)  # 어휘사전 객체 생성
print(tokenizer.encode(text))  # 인코딩 (1131:unk, 1130:endoftext)

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]


In [40]:
print(tokenizer.decode(tokenizer.encode(text)))  # 디코딩, 모르는 단어는 <|unk|> 토큰으로 반환됨

<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.


## 2.5 바이트 페어 인코딩

In [41]:
from importlib.metadata import version
import tiktoken
print("tiktoken 버전:", version("tiktoken"))

tiktoken 버전: 0.12.0


In [42]:
tokenizer = tiktoken.get_encoding("gpt2")  # gpt2 tokenizer 객체 생성

In [43]:
text = (
	"Hello, do you like tea? <|endoftext|> In the sunlit terraces"
	" of someunknownPlace."
)
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})  # token의 id 저장
print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 286, 617, 34680, 27271, 13]


1. BPE 토크나이저는 50,257 크기의 어휘사전을 가지고 있어 <|endoftext|>는 50256번 토큰을 가짐
2. <|unk|> 토큰을 사용하지 않고도 someunknownPlace 같은 처음 보는 단어를 처리할 수 있음

In [44]:
strings = tokenizer.decode(integers)  # token id를 기준으로 다시 string 변경
print(strings)
print(tokenizer.decode([0]))  # 0번 토큰에는 '!'가 있음

Hello, do you like tea? <|endoftext|> In the sunlit terraces of someunknownPlace.
!


## 2.6 슬라이딩 윈도로 데이터 샘플링하기

In [45]:
with open ("the-verdict.txt", "r", encoding="utf-8") as f:
	raw_text = f.read()
	
enc_text = tokenizer.encode(raw_text)  # verdict.txt 소설 전체 토큰화 (tokenizer는 gpt2 어휘 사전 객체)
print(len(enc_text))

5145


In [46]:
enc_sample = enc_text[50:]  # 처음 50개 토큰 삭제

In [47]:
context_size = 4  # 입력에 포함할 토큰 크기
x = enc_sample[:context_size]  # 입력 토큰을 담은 x
y = enc_sample[1:context_size+1]  # 입력에서 토큰 하나만큼 이동 y
print(f"x: {x}")
print(f"y:      {y}")

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


In [48]:
# 다음 단어 예측
for i in range(1, context_size+1):
	context = enc_sample[:i]  # LLM이 받을 입력
	desired = enc_sample[i]  # LLM이 예측해야 할 타깃 토큰 ID
	print(context, "--->", desired)

[290] ---> 4920
[290, 4920] ---> 2241
[290, 4920, 2241] ---> 287
[290, 4920, 2241, 287] ---> 257


In [49]:
for i in range(1, context_size+1):
	context = enc_sample[:i]
	desired = enc_sample[i]
	print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))  # 토큰 ID를 텍스트로 변경해서 출력해보기

 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a


In [50]:
import torch
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
	def __init__(self, txt, tokenizer, max_length, stride):
		self.input_ids = []
		self.target_ids = []

		token_ids = tokenizer.encode(txt)  # 전체 txt 토큰화

		# 슬라이딩 윈도 사용
		for i in range(0, len(token_ids) - max_length, stride):  # 책을 max_length 길이의 중첩된 시퀀스로 분할
			input_chunk = token_ids[i:i + max_length]  # LLM이 받을 입력
			target_chunk = token_ids[i + 1: i + max_length + 1]  # LLM이 예측해야 할 타깃 토큰
			self.input_ids.append(torch.tensor(input_chunk))  # 입력 텐서
			self.target_ids.append(torch.tensor(target_chunk))  # 타깃 텐서

	def __len__(self):  # 데이터셋에 있는 전체 행 수 반환
		return len(self.input_ids)
	
	def __getitem__(self, idx):  # 데이터셋에서 하나의 행 반환
		return self.input_ids[idx], self.target_ids[idx]

In [ ]:
def create_dataloder_v1(txt, batch_size=4, max_length=256, stride=128, shuffle=True, drop_last=True, num_workers=0):
    tokenizer = tiktoken.get_encoding("gpt2")  # 토크나이저 초기화
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)  # 데이터셋 생성
    dataloader = DataLoader (
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,  # True로 지정하면 batch_size보다 작을 경우 훈련 손실이 갑자기 높아지는 것을 피하기 위해 마지막 배치 삭제
        num_workers=num_workers  # 전처리에 사용할 CPU 프로세서 개수
    )

    return dataloader

In [ ]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

dataloader = create_dataloder_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False)  # 문맥크기 4, 배치 크기 1
data_iter = iter(dataloader)  # 데이터 로더를 파이썬 반복자로 변환한 다음, 파이썬 next() 함수로 다음 원소 추출
first_batch = next(data_iter)
print(first_batch)  # 첫 번째 텐서는 입력 토큰 ID저장, 두 번째 텐서는 타깃 토큰 ID 저장

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [ ]:
second_batch = next(data_iter)
print(second_batch)  # stride를 1로 지정해서 1씩 밀림

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


### 연습문제 2.2

In [58]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

dataloader = create_dataloder_v1(
    raw_text, batch_size=1, max_length=2, stride=2, shuffle=False)  # 문맥크기 2, stride 2
data_iter = iter(dataloader)  # 데이터 로더를 파이썬 반복자로 변환한 다음, 파이썬 next() 함수로 다음 원소 추출
first_batch = next(data_iter)
print(first_batch)  # 첫 번째 텐서는 입력 토큰 ID저장, 두 번째 텐서는 타깃 토큰 ID 저장

[tensor([[ 40, 367]]), tensor([[ 367, 2885]])]


In [59]:
second_batch = next(data_iter)
print(second_batch)  # stride를 2로 지정해서 2씩 밀림

[tensor([[2885, 1464]]), tensor([[1464, 1807]])]


In [119]:
dataloader = create_dataloder_v1(
    raw_text, batch_size=1, max_length=8, stride=2, shuffle=False)  # 문맥크기 2, stride 2
data_iter = iter(dataloader)  # 데이터 로더를 파이썬 반복자로 변환한 다음, 파이썬 next() 함수로 다음 원소 추출
first_batch = next(data_iter)
print(first_batch)  # 첫 번째 텐서는 입력 토큰 ID저장, 두 번째 텐서는 타깃 토큰 ID 저장

[tensor([[  40,  367, 2885, 1464, 1807, 3619,  402,  271]]), tensor([[  367,  2885,  1464,  1807,  3619,   402,   271, 10899]])]


In [110]:
second_batch = next(data_iter)
print(second_batch)  # stride를 2로 지정해서 2씩 밀림

[tensor([[ 2885,  1464,  1807,  3619,   402,   271, 10899,  2138]]), tensor([[ 1464,  1807,  3619,   402,   271, 10899,  2138,   257]])]


In [ ]:
dataloader = create_dataloder_v1 (
    raw_text, batch_size=8, max_length=4, stride=1, shuffle = False  # stride로 입력 배치 내에서의 sliding 크기를 정함
)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("입력:\n", inputs)
print("\n타깃\n", targets)

입력:
 tensor([[   40,   367,  2885,  1464],
        [  367,  2885,  1464,  1807],
        [ 2885,  1464,  1807,  3619],
        [ 1464,  1807,  3619,   402],
        [ 1807,  3619,   402,   271],
        [ 3619,   402,   271, 10899],
        [  402,   271, 10899,  2138],
        [  271, 10899,  2138,   257]])

타깃
 tensor([[  367,  2885,  1464,  1807],
        [ 2885,  1464,  1807,  3619],
        [ 1464,  1807,  3619,   402],
        [ 1807,  3619,   402,   271],
        [ 3619,   402,   271, 10899],
        [  402,   271, 10899,  2138],
        [  271, 10899,  2138,   257],
        [10899,  2138,   257,  7026]])


In [121]:
second_inputs, second_targets = next(data_iter)
print("입력:\n", second_inputs)
print("\n타깃\n", second_targets)

입력:
 tensor([[10899,  2138,   257,  7026],
        [ 2138,   257,  7026, 15632],
        [  257,  7026, 15632,   438],
        [ 7026, 15632,   438,  2016],
        [15632,   438,  2016,   257],
        [  438,  2016,   257,   922],
        [ 2016,   257,   922,  5891],
        [  257,   922,  5891,  1576]])

타깃
 tensor([[ 2138,   257,  7026, 15632],
        [  257,  7026, 15632,   438],
        [ 7026, 15632,   438,  2016],
        [15632,   438,  2016,   257],
        [  438,  2016,   257,   922],
        [ 2016,   257,   922,  5891],
        [  257,   922,  5891,  1576],
        [  922,  5891,  1576,   438]])


## 2.7 토큰 임베딩 만들기

In [124]:
input_ids = torch.tensor([2, 3, 5, 1])
vocab_size=6
output_dim = 3

In [ ]:
torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)  # 6개의 토큰에 각각 하나의 행 할당, 3개의 임베딩 차원에 각각 하나의 열 할당
print(embedding_layer.weight)  # 임베딩 층에 있는 가중치 출력

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


In [ ]:
print(embedding_layer(torch.tensor([3])))  # 임베딩 층은 토큰 ID를 기반으로 가중치 행렬에서 행을 추출하는 검색 연산 수행

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


In [ ]:
print(embedding_layer(input_ids))  # torch.tensor([2, 3, 5, 1])
# embedding_layer에서 2, 3, 5, 1번째 행 출력

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


## 2.8 단어 위치 인코딩하기